In [2]:
# Código para a extração, segmentação e predição da turbidez de uma área específica.

import ee
import geemap
import joblib
import numpy as np
import os
import threading
from threading import Lock

# Inicializa o Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# Define a Área de Interesse (AOI)
aoi = ee.Geometry.Polygon([[[-45.559114, -18.954365], [-45.559114, -18.212409], 
                            [-44.839706, -18.212409], [-44.839706, -18.954365], 
                            [-45.559114, -18.954365]]])


# Carrega as imagens do Sentinel-2
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(aoi) \
    .filterDate('2023-01-01', '2023-09-01') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(lambda image: image.clip(aoi))

# Seleciona as bandas de interesse
bands = ['B2', 'B3', 'B4', 'B5', 'B8']
image = sentinel2.select(bands).median()

# Calcula o NDWI para criar a máscara de água
NDWI = image.expression('((B3 - B8) / (B3 + B8))', {'B3': image.select('B3'), 'B8': image.select('B8')})

# Aplica a máscara de água
water_mask = NDWI.gt(0).rename('water_mask')
water_only_image = image.updateMask(water_mask)

# Calcula índices para pixels de água
ITBDN = water_only_image.expression('((B3 - B2) / (B3 + B2))', {'B2': water_only_image.select('B2'), 'B3': water_only_image.select('B3')})
RB3B2 = water_only_image.expression('(B3 / B2)', {'B2': water_only_image.select('B2'), 'B3': water_only_image.select('B3')})

# Carrega o modelo treinado
model = joblib.load('models/Modelo3Marias.sav')



# Extrai os coeficientes do modelo
coefficients = model.coef_
intercept = model.intercept_

# Parâmetros de normalização
median = 3.92  # mediana dos dados de treinamento
iqr = 2.52  # intervalo interquartil (IQR) dos dados de teste



# Constantes do Earth Engine
median_ee = ee.Number(median)
iqr_ee = ee.Number(iqr)

# Define a expressão de previsão de turbidez
expr_B2 = water_only_image.select('B2').subtract(median_ee).divide(iqr_ee).multiply(coefficients[0])
expr_B3 = water_only_image.select('B3').subtract(median_ee).divide(iqr_ee).multiply(coefficients[1])
expr_B4 = water_only_image.select('B4').subtract(median_ee).divide(iqr_ee).multiply(coefficients[2])
expr_B5 = water_only_image.select('B5').subtract(median_ee).divide(iqr_ee).multiply(coefficients[3])
expr_ITBDN = ITBDN.subtract(median_ee).divide(iqr_ee).multiply(coefficients[4])
expr_RB3B2 = RB3B2.subtract(median_ee).divide(iqr_ee).multiply(coefficients[5])
expr_NDWI = NDWI.subtract(median_ee).divide(iqr_ee).multiply(coefficients[6])

# Imagem final de previsão de turbidez
predicted_image = expr_B2.add(expr_B3).add(expr_B4).add(expr_B5).add(expr_ITBDN).add(expr_RB3B2).add(expr_NDWI).add(intercept).rename('turbidez_pred')

# Diretório para salvar os arquivos TIFF
save_directory = 'analises_turbidez/3marias'  # Altere para o caminho desejado
os.makedirs(save_directory, exist_ok=True)  # Cria o diretório se não existir

# Função para dividir a AOI e exportar imagens em paralelo
def split_aoi_and_export(aoi, n_tiles, scale, image, lock, tile_list):
    """
    Divide a AOI em uma grade NxN e exporta as imagens em paralelo.

    Parâmetros:
    aoi (Geometry): Área de Interesse.
    n_tiles (int): Número de tiles em cada direção.
    scale (int): Escala da exportação.
    image (Image): Imagem a ser exportada.
    lock (Lock): Lock para controle de acesso ao recurso compartilhado.
    tile_list (list): Lista para armazenar as imagens exportadas.
    """
    aoi_bounds = aoi.bounds().coordinates().getInfo()[0]
    xmin, ymin = aoi_bounds[0][0], aoi_bounds[0][1]
    xmax, ymax = aoi_bounds[2][0], aoi_bounds[2][1]
    x_step = (xmax - xmin) / n_tiles
    y_step = (ymax - ymin) / n_tiles

    def export_tile(i, j):
        """
        Exporta um tile específico da AOI.

        Parâmetros:
        i (int): Índice do tile na direção x.
        j (int): Índice do tile na direção y.
        """
        x0 = xmin + i * x_step
        x1 = xmin + (i + 1) * x_step
        y0 = ymin + j * y_step
        y1 = ymin + (j + 1) * y_step
        tile = ee.Geometry.Polygon([[[x0, y0], [x1, y0], [x1, y1], [x0, y1], [x0, y0]]])
        tile_image = image.clip(tile)
        tile_list.append(tile_image)
        out_file = os.path.join(save_directory, f'PredictedTurbidity_Tile_{i+1}_{j+1}.tif')
        lock.acquire()
        try:
            geemap.ee_export_image(tile_image, filename=out_file, scale=scale, region=tile)
            print(f'Imagem salva localmente: {out_file}')
        finally:
            lock.release()

    threads = []
    lock = Lock()
    for i in range(n_tiles):
        for j in range(n_tiles):
            t = threading.Thread(target=export_tile, args=(i, j))
            threads.append(t)
            t.start()

    for t in threads:
        t.join()

# Exporta os tiles
tile_list = []
n_tiles = 2  # Grade NxN
lock = Lock()
split_aoi_and_export(aoi, n_tiles, scale=30, image=predicted_image, lock=lock, tile_list=tile_list)


# ---- Seção 2: Recuperar os Tiles, Mesclar e Montar o Mapa ----

import ee
import geemap

# Inicializa o Earth Engine (se necessário)
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# Mescla os tiles usando mosaico
merged_image = ee.ImageCollection(tile_list).mosaic()

# Calcula os valores mínimo e máximo de turbidez após a mesclagem
min_max_values = merged_image.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
).getInfo()

min_value = min_max_values['turbidez_pred_min']
max_value = min_max_values['turbidez_pred_max']

# Imprime os valores mínimo e máximo calculados
print(f"Valor Mínimo de Turbidez: {min_value}")
print(f"Valor Máximo de Turbidez: {max_value}")

# Exibe a imagem mesclada no mapa
Map = geemap.Map()
Map.centerObject(aoi, zoom=10)
Map.add_basemap('SATELLITE')

# Parâmetros de visualização
vis_params = {
    'min': min_value,
    'max': max_value,
    'palette': [
        'blue', 'cyan', 'green', 'yellow', 'orange', 'red',
        'darkred', 'purple', 'magenta', 'brown', 'black'
    ]
}
Map.addLayer(merged_image, vis_params, 'Turbidez Predita Mesclada')
Map.addLayer(aoi, {}, 'Limite da AOI')
Map.addLayerControl()

# Mostra o mapa
Map


e:\Projetos\main\tempEnv\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator HuberRegressor from version 1.5.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_turbidez\3marias\PredictedTurbidity_Tile_1_1.tif
Imagem salva localmente: analises_turbidez/3marias\PredictedTurbidity_Tile_1_1.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_turbidez\3marias\PredictedTurbidity_Tile_1_2.tif
Imagem salva localmente: analises_turbidez/3marias\PredictedTurbidity_Tile_1_2.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_turbidez\3marias\PredictedTurbidity_Tile_2_1.tif
Imagem salva localmente: analises_turbidez/3marias\PredictedTurbidity_Tile_2_1.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_turbidez\3marias\PredictedTurbidity_Tile_2_2.tif
Imagem salva localmente: analises_turbidez/3marias\PredictedTurbidity_Tile_2_2.tif
Valor Mínimo de Turbidez: 284.81552051632536
Valor Máximo de Turbidez: 835.0553724784698


Map(center=[-18.58345884758661, -45.1994100000001], controls=(WidgetControl(options=['position', 'transparent_…

In [1]:


import ee
import geemap
import joblib
import numpy as np
import os
import threading
from threading import Lock


try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# (AOI)
aoi = ee.Geometry.Polygon([[[-45.559114, -18.954365], [-45.559114, -18.212409], 
                            [-44.839706, -18.212409], [-44.839706, -18.954365], 
                            [-45.559114, -18.954365]]])

# Sentinel-2
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(aoi) \
    .filterDate('2020-01-01', '2020-04-01') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(lambda image: image.clip(aoi))

# Seleciona as bandas de interesse
bands = ['B2', 'B3', 'B4', 'B5', 'B8']
image = sentinel2.select(bands).median()

# Calcula o NDWI para criar a máscara de água
NDWI = image.expression('((B3 - B8) / (B3 + B8))', {'B3': image.select('B3'), 'B8': image.select('B8')})

# Aplica a máscara de água
water_mask = NDWI.gt(0).rename('water_mask')
water_only_image = image.updateMask(water_mask)

# Calcula índices para pixels de água
ITBDN = water_only_image.expression('((B3 - B2) / (B3 + B2))', {'B2': water_only_image.select('B2'), 'B3': water_only_image.select('B3')})
RB3B2 = water_only_image.expression('(B3 / B2)', {'B2': water_only_image.select('B2'), 'B3': water_only_image.select('B3')})

# Carrega o modelo treinado
model = joblib.load('models/Modelo3Marias.sav')

# Extrai os coeficientes do modelo
coefficients = model.coef_
intercept = model.intercept_

# Parâmetros de normalização
median = 3.92  # mediana dos dados de treinamento
iqr = 2.52  # intervalo interquartil (IQR) dos dados de teste

# Constantes do Earth Engine
median_ee = ee.Number(median)
iqr_ee = ee.Number(iqr)

# Define a expressão de previsão de turbidez
expr_B2 = water_only_image.select('B2').subtract(median_ee).divide(iqr_ee).multiply(coefficients[0])
expr_B3 = water_only_image.select('B3').subtract(median_ee).divide(iqr_ee).multiply(coefficients[1])
expr_B4 = water_only_image.select('B4').subtract(median_ee).divide(iqr_ee).multiply(coefficients[2])
expr_B5 = water_only_image.select('B5').subtract(median_ee).divide(iqr_ee).multiply(coefficients[3])
expr_ITBDN = ITBDN.subtract(median_ee).divide(iqr_ee).multiply(coefficients[4])
expr_RB3B2 = RB3B2.subtract(median_ee).divide(iqr_ee).multiply(coefficients[5])
expr_NDWI = NDWI.subtract(median_ee).divide(iqr_ee).multiply(coefficients[6])

# Imagem final de previsão de turbidez
predicted_image = expr_B2.add(expr_B3).add(expr_B4).add(expr_B5).add(expr_ITBDN).add(expr_RB3B2).add(expr_NDWI).add(intercept).rename('turbidez_pred')

# Diretório para salvar os arquivos TIFF
save_directory = 'analises_turbidez/3marias'  # Altere para o caminho desejado
os.makedirs(save_directory, exist_ok=True)  # Cria o diretório se não existir

# Função para dividir a AOI e exportar imagens em paralelo
def split_aoi_and_export(aoi, n_tiles, scale, image, lock, tile_list):
    """
    Divide a AOI em uma grade NxN e exporta as imagens em paralelo.

    Parâmetros:
    aoi (Geometry): Área de Interesse.
    n_tiles (int): Número de tiles em cada direção.
    scale (int): Escala da exportação.
    image (Image): Imagem a ser exportada.
    lock (Lock): Lock para controle de acesso ao recurso compartilhado.
    tile_list (list): Lista para armazenar as imagens exportadas.
    """
    aoi_bounds = aoi.bounds().coordinates().getInfo()[0]
    xmin, ymin = aoi_bounds[0][0], aoi_bounds[0][1]
    xmax, ymax = aoi_bounds[2][0], aoi_bounds[2][1]
    x_step = (xmax - xmin) / n_tiles
    y_step = (ymax - ymin) / n_tiles

    def export_tile(i, j):
        """
        Exporta um tile específico da AOI.

        Parâmetros:
        i (int): Índice do tile na direção x.
        j (int): Índice do tile na direção y.
        """
        x0 = xmin + i * x_step
        x1 = xmin + (i + 1) * x_step
        y0 = ymin + j * y_step
        y1 = ymin + (j + 1) * y_step
        tile = ee.Geometry.Polygon([[[x0, y0], [x1, y0], [x1, y1], [x0, y1], [x0, y0]]])
        tile_image = image.clip(tile)
        tile_list.append(tile_image)
        out_file = os.path.join(save_directory, f'PredictedTurbidity_Tile_{i+1}_{j+1}.tif')
        lock.acquire()
        try:
            geemap.ee_export_image(tile_image, filename=out_file, scale=scale, region=tile)
            print(f'Imagem salva localmente: {out_file}')
        finally:
            lock.release()

    threads = []
    lock = Lock()
    for i in range(n_tiles):
        for j in range(n_tiles):
            t = threading.Thread(target=export_tile, args=(i, j))
            threads.append(t)
            t.start()

    for t in threads:
        t.join()

# Exporta os tiles
tile_list = []
n_tiles = 2  # Grade NxN
lock = Lock()
split_aoi_and_export(aoi, n_tiles, scale=30, image=predicted_image, lock=lock, tile_list=tile_list)

# ---- Seção 2: Recuperar os Tiles, Mesclar e Montar o Mapa ----

# Inicializa o Earth Engine (se necessário)
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# Mescla os tiles usando mosaico
merged_image = ee.ImageCollection(tile_list).mosaic()

# Calcula os valores mínimo e máximo de turbidez após a mesclagem
min_max_values = merged_image.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
).getInfo()

min_value = min_max_values['turbidez_pred_min']
max_value = min_max_values['turbidez_pred_max']

# Imprime os valores mínimo e máximo calculados
print(f"Valor Mínimo de Turbidez: {min_value}")
print(f"Valor Máximo de Turbidez: {max_value}")

# Exibe a imagem mesclada no mapa
Map = geemap.Map()
Map.centerObject(aoi, zoom=10)
Map.add_basemap('SATELLITE')

# Parâmetros de visualização
vis_params = {
    'min': min_value,
    'max': max_value,
    'palette': [
        'blue', 'cyan', 'green', 'yellow', 'orange', 'red',
        'darkred', 'purple', 'magenta', 'brown', 'black'
    ]
}
Map.addLayer(merged_image, vis_params, 'Turbidez Predita Mesclada')
Map.addLayer(aoi, {}, 'Limite da AOI')
Map.addLayerControl()


# Função para adicionar a legenda
def add_legend(map_obj, title, palette, min_value, max_value):
    """
    Adiciona uma legenda ao mapa.

    Parâmetros:
    map_obj (geemap.Map): O objeto de mapa geemap.
    title (str): O título da legenda.
    palette (list): A paleta de cores usada para o mapa.
    min_value (float): O valor mínimo da escala.
    max_value (float): O valor máximo da escala.
    """
    # Definindo a legenda em HTML
    legend_html = f"""
    <div style='padding: 10px; background-color: white; border-radius: 5px;'>
        <h4>{title}</h4>
        <div style='display: flex; align-items: center;'>
            <span>low</span>
            <div style='flex-grow: 1; height: 20px; background: linear-gradient(to right, {", ".join(palette)}); margin: 0 10px;'></div>
            <span>high</span>
        </div>
    </div>
    """
    map_obj.add_html(legend_html)

# Adiciona a legenda ao mapa
add_legend(Map, 'Turbidez Predita (NTU)', vis_params['palette'], min_value, max_value)


# Mostra o mapa
Map


e:\Projetos\main\tempEnv\Lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)
e:\Projetos\main\tempEnv\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator HuberRegressor from version 1.5.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_turbidez\3marias\PredictedTurbidity_Tile_1_1.tif
Imagem salva localmente: analises_turbidez/3marias\PredictedTurbidity_Tile_1_1.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_turbidez\3marias\PredictedTurbidity_Tile_1_2.tif
Imagem salva localmente: analises_turbidez/3marias\PredictedTurbidity_Tile_1_2.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_turbidez\3marias\PredictedTurbidity_Tile_2_1.tif
Imagem salva localmente: analises_turbidez/3marias\PredictedTurbidity_Tile_2_1.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_turbidez\3marias\PredictedTurbidity_Tile_2_2.tif
Imagem salva localmente: analises_turbidez/3marias\PredictedTurbidity_Tile_2_2.tif
Valor Mínimo de Turbidez: 284.81552051632536
Valor Máximo de Turbidez: 835.0553724784698


Map(center=[-18.58345884758661, -45.1994100000001], controls=(WidgetControl(options=['position', 'transparent_…